# DFT + GWBSE Energy Calculation Using CH4

## Introduction

This tutorial explains how to perform calculation to predict electronic excitation using the **GWBSE** method. See [the GW Compendium: A Practical Guide to Theoretical Photoemission Spectroscopy](https://doi.org/10.3389/fchem.2019.00377), for an excellent introduction to the method.

## Requirements
* You will need to install **VOTCA** using the instructions described [here](https://github.com/votca/votca/blob/master/share/sphinx/INSTALL.rst)
* Once the installation is completed you need to activate the VOTCA enviroment by running the `VOTCARC.bash` script that has been installed at the bin subfolder for the path that you have provided for the installation step above

## Interacting with the XTP command line interface
To run a DFT-GWBSE calculation we will use the [xtp_tools](https://www.votca.org/xtp/xtp_tools_overview.html) calculator. Run the following command to view the help message of `xtp_tools`:

In [ ]:
!xtp_tools --help

### Note
> * In Jupyter the `!` symbol means: *run the following command as a standard unix command*
> * In Jupyter the command `%env` set an environmental variable

## Running a calculation with the default options
To run a DFT-GWBSE calculation we just need to provide the path to the file in XYZ with the molecular coordinates. Check the [dftgwbse defaults](https://www.votca.org/xtp/dftgwbse.html) for further information.

In [ ]:
!xtp_tools -c job_name=methane -t 2 -e dftgwbse > dftgwbse.log

The previous command will run the DFT-GWBSE calculation using the aforementioned defaults and the results are store in the *Current Work Directory* in a file named `methane_summary.xml`. The `-c` option is important and we will come back to it later. It allows changing options form the command line.

## Running a calculation using your own input file
Let create a folder to store the input `options` for XTP and use the `-p` option to print an option file, specified by `-o`, with all the options so we can modify it afterwards

In [ ]:
!mkdir -p OPTIONFILES
!xtp_tools -p dftgwbse -o OPTIONFILES/dftgwbse.xml

You should have a *XML* file with the DFTWGSE options that looks like

In [ ]:
!head -n 10 OPTIONFILES/dftgwbse.xml

Some options are labelled as `OPTIONAL`, either fill them in or delete them if you do not want that functionality

We created a small options file

In [ ]:
!cat dftgwbse2.xml

In [ ]:
!xtp_tools -o dftgwbse2.xml -t 2 -e dftgwbse > dftgwbse2.log

XTP will automatically compare the default values with the user-provided and overwrites the defaults with the user input. Also, If I given property does not have a default value you can provide one using the XML file described above.

## Partial Charges
We can compute now the partial charges using the `CHELPG` method by default. For more information see the [partialcharges documentation](https://www.votca.org/xtp/partialcharges.html). Once again, we only need to provide the name of the system to compute, which in our case is `methane`.

In [ ]:
!xtp_tools -c job_name=methane -e partialcharges

## Spectrum Calculation
Finally, lets compute a convolution of the singlet spectrum using a gaussian function. For doing so, we will modify the default values for the [spectrum calculator](https://www.votca.org/xtp/spectrum.html) to compute the spectrum between 9 and 25 eV, using 1000 points in that energy range. We will use the `-c` option to modify the options accordingly. Instead we could have printed out an options file using the `xtp_tools -p spectrum` command and then modify the entries accordingly and then read them in using the `-o` option.

In [ ]:
!xtp_tools -c job_name=methane lower=9 upper=25 points=1000 -e spectrum

The results are stored in the `methane_spectrum.dat` file.

## (Optional) Plot the spectrum

We will use [matplotlib](https://matplotlib.org/), [seaborn](https://seaborn.pydata.org/) and [pandas](https://pandas.pydata.org/) libraries to plot the spectrum. You can install it using [pip](https://pip.pypa.io/en/stable/) like

In [ ]:
!pip install seaborn --user

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
columns = ["E(eV)", "epsGaussian","IM(eps)Gaussian",  "epsLorentz",  "Im(esp)Lorentz"] 
df = pd.read_table("methane_spectrum.dat", comment="#", sep='\s+',names=columns) 
sns.relplot(x="E(eV)", y="epsGaussian", ci=None, kind="line", data=df) 
plt.plot()